In [1]:
# Load image
# Get their 3 image stars most brightness
# Get the distances: a->b, a->c, b->c
# Load the catalog
# Get pairs in the distance ab+-0.1 and save them as S1
# Get pairs in the distance ac+-0.1 and save them as S2
# For each star in both S1 and S2 check the other two stars angular distance
# it should be equal to ac+-0.1

In [2]:
import cv2
import numpy as np
from image_star import ImageStar, Centroid
from catalog import Catalog



In [3]:
def compute_i_border(cX, cY, ROI, img):
    offset = int((ROI-1)/2)
    x_start = cX - offset # 0
    x_end = cX + offset # 4
    y_start = cY - offset # 0
    y_end = cY + offset # 4
    i_border = 0

    if x_start < 0 or x_end >= img.shape[1]:
        return i_border
    if y_start < 0 or y_end >= img.shape[0]:
        return i_border

    for x in range(x_start, x_end+1):
        i_border += img[y_start][x] # Top border
        i_border += img[y_end][x] # Bottom border

    for y in range(y_start+1, y_end):
        i_border += img[y][x_start] # Left border
        i_border += img[y][x_end] # Right border

    return i_border/(4*(ROI-1))


def compute_i_brightness(cX, cY, i_border, ROI, img):
    offset = int((ROI-1) / 2 - 1)
    x_start = cX - offset
    x_end = cX + offset
    y_start = cY - offset
    y_end = cY + offset
    B = 0

    if x_start < 0 or x_end > img.shape[1]:
        return B
    if y_start < 0 or y_end > img.shape[0]:
        return B

    for x in range(x_start, x_end+1):
        for y in range(y_start, y_end+1):
            pixel_value = img[y][x]
            B += pixel_value - i_border
    x_cm = 0.0
    y_cm = 0.0
    if B != 0:
        for x in range(x_start, x_end+1):
            for y in range(y_start, y_end+1):
                pixel_value = img[y][x] - i_border
                x_cm = x_cm + (x*pixel_value)/B
                y_cm = y_cm + (y*pixel_value)/B
    else:
        x_cm = x_start + (x_end-x_start)/2
        y_cm = y_start + (y_end-y_start)/2

    return [x_cm, y_cm, B]


def simplify_img_stars(stars, thresh1=9, thresh2=4):
    simplified = []

    for i in range(0, len(stars)):
        suma_x = 0
        suma_y = 0
        suma_B = 0
        if stars[i].centroid.x != 2000 and stars[i].centroid.y != 2000:
            new_x = stars[i].centroid.x
            new_y = stars[i].centroid.y
            new_B = stars[i].brightness
            pixels = 1

            suma_x = new_x
            suma_y = new_y
            suma_B = new_B

            for j in range(i+1, len(stars)):
                if abs(new_x - stars[j].centroid.x) <= thresh1 and abs(new_y - stars[j].centroid.y) <= thresh1:
                    suma_x += stars[j].centroid.x
                    suma_y += stars[j].centroid.y
                    suma_B += stars[j].brightness
                    pixels += 1
                    stars[j].centroid.x = 2000
                    stars[j].centroid.y = 2000

            stars[i].centroid.x = 2000
            stars[i].centroid.y = 2000
            if pixels > thresh2:
                centroid = Centroid(suma_x/pixels, suma_y/pixels)
                img_star = ImageStar(centroid, suma_B/pixels)
                simplified.append(img_star)

    return simplified


In [103]:
image = cv2.imread("./test_images/vega_05sec.jpg")
# Grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [104]:
img_stars = []
# Step 1: Get possible Image stars, these are possible stars in the image.
# Define ROI, must be odd number
my_ROI = 9
limit = int((my_ROI - 1) / 2)
thresh_min_value = 80
for x in range(limit, gray.shape[1]-limit):
    for y in range(limit, gray.shape[0]-limit):
        if gray[y][x] >= thresh_min_value:
            my_i_border = compute_i_border(x, y, my_ROI, gray)
            my_x_cm, my_y_cm, my_B = compute_i_brightness(x, y, my_i_border, my_ROI, gray)
            centroid = Centroid(my_x_cm, my_y_cm)
            img_star = ImageStar(centroid, my_B)
            img_stars.append(img_star)

img_stars = simplify_img_stars(img_stars)
# Sort image_stars by brightness
img_stars = sorted(img_stars, key=lambda x: x.brightness, reverse=True)
# Print the thre most brightness stars selected
for i in range(0,3):
    print(img_stars[i].centroid.x, img_stars[i].centroid.y, img_stars[i].brightness)

1142.852130425894 603.109172475432 3172.407738095238
1239.9489682160768 776.0931532642597 2681.3459821428573
496.6762730980482 392.0364623002791 2323.4523026315787


In [105]:
# Compute unitary vectors
unitaries = []
for img_star in img_stars:
    unitaries.append(img_star.get_unitary_vector())


In [106]:
# load catalogs
catalog = Catalog("./generation/out/hip_2020.csv",
                  "./generation/out/guide_stars_2020_4.5.csv",
                  "./generation/out/guide_stars_neighboors_4.5.json")


In [107]:
star1 = img_stars[0]
star2 = img_stars[1]
star3 = img_stars[2]


d_star1_star2 = star1.get_distance(star2)
d_star1_star3 = star1.get_distance(star3)
d_star2_star3 = star2.get_distance(star3)

print(f'{d_star1_star2=} {d_star1_star3=} {d_star2_star3=}')


d_star1_star2=4.116234805246254 d_star1_star3=14.515775341278863 d_star2_star3=17.742570176274008


In [108]:
# Get pairs in distance star1->star2 (11.800359100347473) +- 0.1
pairs_a_b = catalog.get_pairs_distance(d_star1_star2-0.1, d_star1_star2+0.1)
pairs_a_c = catalog.get_pairs_distance(d_star1_star3-0.1, d_star1_star3+0.1)

In [109]:
for pair1 in pairs_a_b:
    for pair2 in pairs_a_c:
        if pair1[0] == pair2[0]:
            hip_x = pair1[1]
            hip_y = pair2[1]
        elif pair1[0] == pair2[1]:
            hip_x = pair1[1]
            hip_y = pair2[0]
        elif pair1[1] == pair2[0]:
            hip_x = pair1[0]
            hip_y = pair2[1]
        elif pair1[1] == pair2[1]:
            hip_x = pair1[0]
            hip_y = pair2[0]
        else:
            continue
            
        star_x = catalog.get_star_by_id(hip_x)
        star_y = catalog.get_star_by_id(hip_y)
        other_distance = star_x.get_distance(star_y)
        if other_distance > d_star2_star3-0.1 and other_distance < d_star2_star3+0.1:
            print(f'Triplet found:')
            print(f'{pair1=} {pair2=} pair3=[{hip_x}, {hip_y}, {other_distance}')
            

Triplet found:
pair1=[85670, 87833, 4.117412872783996] pair2=[87833, 91262, 14.528670955016638] pair3=[85670, 91262, 17.75470365207743


In [110]:
# try again with star2 as kernel
star1 = img_stars[0]
star2 = img_stars[1]
star3 = img_stars[2]


d_star2_star1 = star2.get_distance(star1)
d_star2_star3 = star2.get_distance(star3)
d_star1_star3 = star1.get_distance(star3)

print(f'{d_star2_star1=} {d_star2_star3=} {d_star1_star3=}')

d_star2_star1=4.116234805246254 d_star2_star3=17.742570176274008 d_star1_star3=14.515775341278863


In [111]:
pairs_b_a = catalog.get_pairs_distance(d_star2_star1-0.1, d_star2_star1+0.1)
pairs_b_c = catalog.get_pairs_distance(d_star2_star3-0.1, d_star2_star3+0.1)

In [112]:
for pair1 in pairs_b_a:
    for pair2 in pairs_b_c:
        if pair1[0] == pair2[0]:
            hip_x = pair1[1]
            hip_y = pair2[1]
        elif pair1[0] == pair2[1]:
            hip_x = pair1[1]
            hip_y = pair2[0]
        elif pair1[1] == pair2[0]:
            hip_x = pair1[0]
            hip_y = pair2[1]
        elif pair1[1] == pair2[1]:
            hip_x = pair1[0]
            hip_y = pair2[0]
        else:
            continue
            
        star_x = catalog.get_star_by_id(hip_x)
        star_y = catalog.get_star_by_id(hip_y)
        other_distance = star_x.get_distance(star_y)
        if other_distance > d_star1_star3-0.1 and other_distance < d_star1_star3+0.1:
            print(f'Triplet found:')
            print(f'{pair1=} {pair2=} pair3=[{hip_x}, {hip_y}, {other_distance}')

Triplet found:
pair1=[85670, 87833, 4.117412872783996] pair2=[85670, 91262, 17.75470365207743] pair3=[87833, 91262, 14.528670955016638


In [113]:
# Try again with star3 as kernel
star1 = img_stars[0]
star2 = img_stars[1]
star3 = img_stars[2]


d_star3_star1 = star3.get_distance(star1)
d_star3_star2 = star3.get_distance(star2)
d_star1_star2 = star1.get_distance(star2)

print(f'{d_star3_star1=} {d_star3_star2=} {d_star1_star2=}')

d_star3_star1=14.515775341278863 d_star3_star2=17.742570176274008 d_star1_star2=4.116234805246254


In [114]:
pairs_c_a = catalog.get_pairs_distance(d_star3_star1-0.1, d_star3_star1+0.1)
pairs_c_b = catalog.get_pairs_distance(d_star3_star2-0.1, d_star3_star2+0.1)

In [115]:
for pair1 in pairs_c_a:
    for pair2 in pairs_c_b:
        if pair1[0] == pair2[0]:
            hip_x = pair1[1]
            hip_y = pair2[1]
        elif pair1[0] == pair2[1]:
            hip_x = pair1[1]
            hip_y = pair2[0]
        elif pair1[1] == pair2[0]:
            hip_x = pair1[0]
            hip_y = pair2[1]
        elif pair1[1] == pair2[1]:
            hip_x = pair1[0]
            hip_y = pair2[0]
        else:
            continue
            
        star_x = catalog.get_star_by_id(hip_x)
        star_y = catalog.get_star_by_id(hip_y)
        other_distance = star_x.get_distance(star_y)
        if other_distance > d_star1_star2-0.1 and other_distance < d_star1_star2+0.1:
            print(f'Triplet found:')
            print(f'{pair1=} {pair2=} pair3=[{hip_x}, {hip_y}, {other_distance}')

Triplet found:
pair1=[87833, 91262, 14.528670955016638] pair2=[85670, 91262, 17.75470365207743] pair3=[87833, 85670, 4.117412872783996
